In [5]:
!pip install transformers
!pip install pymupdf
!pip install gdown

In [6]:
import os
import fitz  #PyMuPDF
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import torch

In [12]:
zip_path = "/content/drive/MyDrive/Resume Dataset/modelo_finetuned.zip"

if not os.path.exists(zip_path):
  !gdown --id 1OEz4XdwJv6wL8uftN1WDd2eZ9y40jyRf
  # Descomprimir el ZIP
  !unzip -q /content/modelo_finetuned.zip -d /content/

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1OEz4XdwJv6wL8uftN1WDd2eZ9y40jyRf
From (redirected): https://drive.google.com/uc?id=1OEz4XdwJv6wL8uftN1WDd2eZ9y40jyRf&confirm=t&uuid=25ed5c4d-79b0-4d3b-9fe4-4bd8d5428b21
To: /content/modelo_finetuned.zip
100% 1.51G/1.51G [00:21<00:00, 68.6MB/s]


## Cargar documento pdf (curriculum externo)

In [52]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
print("📄 PDF cargado:", pdf_path)

Saving 13014900.pdf to 13014900.pdf
📄 PDF cargado: 13014900.pdf


In [53]:
import fitz

def extraer_texto_pdf(pdf_path):
    texto = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            texto += page.get_text()

    return texto

texto_pdf = extraer_texto_pdf(pdf_path)
len(texto_pdf)

10560

In [54]:
from transformers import BartTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("/content/modelo_finetuned")
tokenizer = BartTokenizer.from_pretrained("/content/modelo_finetuned")

In [55]:
def resumir(texto, max_len=300, min_len=40):
    partes = [texto[i:i+3000] for i in range(0, len(texto), 3000)]

    res_final = ""
    for parte in partes:
        inputs = tokenizer([parte], max_length=1024, truncation=True, return_tensors="pt")
        summary_ids = model.generate(
            inputs["input_ids"],
            num_beams=4,
            max_length=max_len,
            min_length=min_len,
            length_penalty=2.0,
            repetition_penalty=2.0,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
        res_final += tokenizer.decode(summary_ids[0], skip_special_tokens=True) + " "

    return res_final.strip()


In [56]:
resumen = resumir(texto_pdf)


In [57]:
import textwrap

print("Generated Summary:")
print(textwrap.fill(resumen, width=80))

Generated Summary:
My career of 34 years includes the graphic art field, fine arts, and elementary
art teacher. My personal evolution and vision helps me to know that I possessing
truth, talent and ability that is unique and highly creative. I am a visual
designer with an aptitude for experimental projects. I utilize the basic
elements of design: color, line, shape, space, texture and value. Dedicated art
professional with over 30 years of hands on experience. Proficient in Adobe
Photoshop, InDesign, Illustrator, Microsoft Word, Excel. Website development and
some HTML. Web maintenance. Part-time in Food and Beverage business for 7 years.
Designed souvenir mugs and t-shirts for large scale tourism distribution.
Clients included Sea World and Busch Gardens. Harrah's, Stratosphere, MGM Grand,
and numerous other Las Vegas attractions. I find everything an opportunity to be
solve problems, and am always coming up with creative solutions. Learned
Photoshop and Microsoft Word. Love gardening